<a href="https://colab.research.google.com/github/shahdhruv29/Big-Data-Processing-Labs/blob/main/labs/Lab3_Spark_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab3: Programming Spark RDD

Dhruv Shah - 202103017

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=61cfb2df5f785a599655efd56eb214a4a08afc04cd1e5690f5d3203f4e0cf3d7
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("SparkDemo").setMaster("local")
sc = SparkContext(conf=conf)

In [ ]:
# Loading the Dataset
input = "/content/gdrive/My Drive/datasets/mr/employee.csv"
lineRDD = sc.textFile(input)
empRDD = lineRDD.map(lambda line: line.split(","))

In [ ]:
#Prints a List one element on a line!
def printList(list_a):
  for e in list_a:
    print (e)

In [ ]:
# #printList( empRDD.collect() )  # prints the full RDD
# printList( empRDD.take(10) )    # print only 10

Q1. Project and print (empno, name) of employees that are from state 'TX' from employee.csv


In [ ]:
tx_employees = empRDD.filter(lambda emp: emp[4] == 'TX')
tx_empno_name = tx_employees.map(lambda emp: (emp[0], emp[1]))
printList(tx_empno_name.collect())

('10012', 'Hector Barbossa')
('10200', 'Anton Chigurh')
('10202', 'Jac McKinzie')


Q2. Generate List of (empno, name, salary, dep_avg_sal) of employees who have salary > 1.5 times of department average from employee.csv

In [ ]:
from statistics import mean
deptSalaryRDD = empRDD.map(lambda x : (x[2], int(x[3])))
deptSalaryMeanMap = deptSalaryRDD.groupByKey().mapValues(mean).collectAsMap()

result = empRDD.filter(lambda x : int(x[3]) > 1.5*deptSalaryMeanMap[x[2]]).map(lambda x : (x[0], x[1], x[3], deptSalaryMeanMap[x[2]]))
printList(result.collect())

('10019', 'Vito Corleone', '170500', 59967.78846153846)
('10015', 'Jason Foss', '178000', 97064.64)
('10272', 'Debra Houlihan', '180000', 68684.0625)
('10288', 'Peter Monroe', '157000', 97064.64)
('10086', 'Katie Roper', '150290', 97064.64)
('10222', 'Ricardo Ruiz', '148999', 97064.64)
('10010', 'Jennifer Zamora', '220450', 97064.64)


Q3. Compute state wise count of employees from employee.csv


In [ ]:
state_count = empRDD.map(lambda emp: (emp[4], 1)).reduceByKey(lambda a, b: a + b)
printList(state_count.collect())

('MA', 276)
('TX', 3)
('CT', 6)
('VA', 1)
('VT', 2)
('AL', 1)
('WA', 1)
('CA', 1)
('OH', 1)
('IN', 1)
('TN', 1)
('NH', 1)
('RI', 1)
('PA', 1)
('CO', 1)
('NY', 1)
('UT', 1)
('GA', 1)
('FL', 1)
('NC', 1)
('KY', 1)
('ID', 1)
('NV', 1)
('MT', 1)
('OR', 1)
('ND', 1)
('AZ', 1)
('ME', 1)


Q4. Compute the Standard Deviation of salary


In [ ]:
std_salary_rdd = empRDD.map(lambda emp: float(emp[3])).stdev()
print("Standard Deviation of salary is:", std_salary_rdd)

Standard Deviation of salary is: 25116.159611505253


Q5. Compute Department wise Standard Deviation of Salary

In [ ]:
from statistics import pstdev     #population standard deviation of a given dataset.
dnoSalaryRDD = empRDD.map(lambda emp : (emp[2], int(emp[3])))
result = dnoSalaryRDD.groupByKey().mapValues(pstdev)    #it derectly computes the sd of the iterable list of salaries after group by is performed
printList(result.collect())

('5', 11420.80077994759)
('3', 32875.83877242374)
('4', 8754.93245490792)
('1', 19722.68484258672)
('6', 20702.734864531638)
('2', 0.0)


Q6. List (empno, dno, name, salary, dept_sal_sd) for employee that are having salary > 1.5 times of SD.

In [ ]:
from statistics import pstdev
dnoSalaryRDD = empRDD.map(lambda emp : (emp[2], int(emp[3])))
deptSalarySDMap = dnoSalaryRDD.groupByKey().mapValues(pstdev).collectAsMap()
salaryRDD = empRDD.map(lambda emp : int(emp[3]))
filteredEmpRDD = empRDD.filter(lambda emp : int(emp[3]) > 1.5*deptSalarySDMap[emp[2]])
result = filteredEmpRDD.map(lambda emp : (emp[0], emp[2], emp[1], int(emp[3]), deptSalarySDMap[emp[2]]))
printList(result.collect())

('10026', '5', 'Wilson K Adinolfi', 62506, 11420.80077994759)
('10084', '3', 'Karthikeyan Ait Sidi', 104437, 32875.83877242374)
('10196', '5', 'Sarah Akinkuolie', 64955, 11420.80077994759)
('10088', '5', 'Trina Alagbe', 64991, 11420.80077994759)
('10069', '5', 'Carol Anderson', 50825, 11420.80077994759)
('10002', '5', 'Linda Anderson', 57568, 11420.80077994759)
('10194', '4', 'Colby Andreola', 95660, 8754.93245490792)
('10062', '5', 'Sam Athwal', 59365, 11420.80077994759)
('10114', '5', 'Linda Bachiochi', 47837, 11420.80077994759)
('10250', '3', 'Alejandro Bacong', 50178, 32875.83877242374)
('10252', '5', 'Rachael Baczenski', 54670, 11420.80077994759)
('10242', '5', 'Thomas Barbara', 47211, 11420.80077994759)
('10012', '3', 'Hector Barbossa', 92328, 32875.83877242374)
('10265', '5', 'Francesco A Barone', 58709, 11420.80077994759)
('10066', '5', 'Nader Barton', 52505, 11420.80077994759)
('10061', '5', 'Norman Bates', 57834, 11420.80077994759)
('10023', '5', 'Kimberly Beak', 70131, 11420

Q7. Compute how much offset each department's average salary from the overall average.


In [ ]:
# This is for the overall average
salary_rdd = empRDD.map(lambda emp: float(emp[3]))
avg_salary = salary_rdd.mean()

#This is for the Department average
dept_avg_salary = empRDD.map(lambda emp: (emp[2], float(emp[3]))).groupByKey().mapValues(lambda vals: sum(vals)/len(vals))
dept_offset = dept_avg_salary.map(lambda dept: (dept[0], dept[1] - avg_salary))
printList(dept_offset.collect())

('5', -9052.896425921419)
('3', 28043.95511254012)
('4', 27131.415112540126)
('1', 3928.31511254012)
('6', -336.62238745988)
('2', 180979.31511254012)


Q8. Computes monthly summary on web access log of Lab01, and compute:

(a) Total number of requests.

(b) Total download size (in Mega Bytes).

It should output: <Year-Month, Number of Requests, Download Size> for every month like Dec-2016, Jan-2017, and so!

In [ ]:
web_log_path = "/content/gdrive/My Drive/datasets/mr/web_access_log.txt"
web_log_rdd = sc.textFile(web_log_path)

filtered_web_rdd = web_log_rdd.filter(lambda x: x.split()[5][1:] == "GET" and x.split()[9] != "-" and x.split()[8] != "304")

monthly_log_web_rdd = filtered_web_rdd.map(lambda x: (x.split()[3][4:7] + "-" + x.split()[3][8:12], int(x.split()[9])))

converting = 1 / 10**6

result = monthly_log_web_rdd.aggregateByKey(
    zeroValue=(0, 0),
    seqFunc=(lambda x, y: (x[0] + 1, x[1] + y * converting)),
    combFunc=(lambda x, y: (x[0] + y[0], x[1] + y[1]))
)

printList(result.collect())

('Dec-2015', (9694, 269.2144590000028))
('Jan-2016', (27291, 1837.5263319999972))
('Feb-2016', (36063, 1287.2130360010342))


Q9. Find the most popular browser (made most requests) from the “web access log” file of lab01.

In [ ]:
# Function to extract browser from the log line
def extract_browser(line):
    parts = line.split('"')
    if len(parts) > 5:
        user_agent = parts[5]
        if "Firefox" in user_agent:
            return "Firefox"
        elif "Chrome" in user_agent:
            return "Chrome"
        elif "Safari" in user_agent and "Chrome" not in user_agent:
            return "Safari"
        elif "MSIE" in user_agent or "Trident" in user_agent:
            return "Internet Explorer"
        elif "Edge" in user_agent:
            return "Edge"
        elif "Opera" in user_agent or "OPR" in user_agent:
            return "Opera"
        else:
            return "Other"
    return "Unknown"

# Map each line to (browser, 1) and reduce by key to count
browser_counts = web_log_rdd.map(extract_browser).map(lambda browser: (browser, 1)).reduceByKey(lambda a, b: a + b)

# Find the browser with the maximum count
most_popular_browser = browser_counts.reduce(lambda a, b: a if a[1] > b[1] else b)

print(f"The most popular browser is {most_popular_browser[0]} with {most_popular_browser[1]} requests.")

The most popular browser is Chrome with 61183 requests.
